In [65]:
import requests


class Speller(object):
    service = 'http://speller.yandex.net/services/spellservice.json/checkText'

    def __init__(self, text, options=None, lang=None, format_text=None):
        self.text = text
        self.options = options
        self.lang = lang
        self.format_text = format_text
        self._answer = None

    def check(self):
        data = {'text': self.text}
        if self.options:
            data['options'] = self.options
        if self.lang:
            data['lang'] = self.lang
        if self.format_text:
            data['format'] = self.format_text
        answer = requests.post(url=self.service, data=data).json()
        return answer

    @property
    def answer(self):
        if self._answer is None:
            self._answer = self.check()
        return self._answer

    @property
    def correct(self):
        return not self.answer

    @property
    def spellsafe(self):
        raise NotImplementedError("Subclasses should implement this!")


class Word(Speller):

    @property
    def variants(self):
        if self.correct:
            return
        return self.answer[0]['s']

    @property
    def spellsafe(self):
        if self.correct:
            return
        return self.variants[0]


class Text(Speller):

    @property
    def spellsafe(self):
        changes = {el['word']: el['s'][0] for el in self.answer if len(el['s']) > 0}
        result = self.text
        for wrong, fixed in changes.items():
            result = result.replace(wrong, fixed)
        return result

    @property
    def errors(self):
        return [el['word'] for el in self.answer]

print(Text('42 is a cUl maagic namber').spellsafe)

42 is a cool magic number


In [44]:
import pandas as pd

In [67]:
data_pos = pd.read_csv('/media/data/nlp/sentiment/ru-mokoron/positive.csv',
                   names=['id', 'date', 'user', 'text', 'sentiment'], index_col=0,
                   sep=';', usecols=[0, 1, 2, 3, 4])
data_neg = pd.read_csv('/media/data/nlp/sentiment/ru-mokoron/negative.csv',
                   names=['id', 'date', 'user', 'text', 'sentiment'], index_col=0,
                   sep=';', usecols=[0, 1, 2, 3, 4])

data = pd.concat([data_pos, data_neg])

In [47]:
data.sample(5)

,date,user,text,sentiment
id,,,,
409345048023539712,1386430439,MissOlens,"@Taisia_or_Tasya дура, ахах, сама иди) я лучше...",1
411978782774276096,1387058370,LisEvelina,"я плачу, ибо соскучилась по своим моделям:((((((",-1
409375138216673280,1386437613,suprinosha,RT @mashynya7: пиццуля с Катюлей :* http://t.c...,1
414395330872872960,1387634520,alien_im_mir,"Я бы в розовом пошла, но обуви к нему нет... Л...",-1
409992912148369408,1386584902,Green_Diva,"RT @FRERARD_MM: мишель,я еще не скинул фотку :...",1


In [68]:
data.shape, data_pos.shape, data_neg.shape

((226834, 4), (114911, 4), (111923, 4))

In [69]:
from tqdm import tqdm

In [66]:
fixed_texts = []

total = len(data)
for idx, line in tqdm(data.iterrows(), total=total, mininterval=1.0):
    fixed_text = Text(line.text).spellsafe
    fixed_texts.append({
        'id': idx,
        'text': fixed_text,
        'original_text': line.text,
        'sentiment': line.sentiment
    })



  0%|          | 0/226834 [00:00<?, ?it/s]

  0%|          | 30/226834 [00:01<2:07:13, 29.71it/s]

  0%|          | 59/226834 [00:02<2:09:28, 29.19it/s]

  0%|          | 90/226834 [00:03<2:07:54, 29.54it/s]

  0%|          | 121/226834 [00:04<2:07:18, 29.68it/s]

  0%|          | 152/226834 [00:05<2:06:26, 29.88it/s]

  0%|          | 182/226834 [00:06<2:06:49, 29.78it/s]

  0%|          | 213/226834 [00:07<2:06:23, 29.88it/s]

  0%|          | 241/226834 [00:08<2:07:53, 29.53it/s]

  0%|          | 273/226834 [00:09<2:07:02, 29.72it/s]

  0%|          | 304/226834 [00:10<2:06:50, 29.77it/s]

  0%|          | 335/226834 [00:11<2:06:31, 29.83it/s]

  0%|          | 365/226834 [00:12<2:06:45, 29.78it/s]

  0%|          | 395/226834 [00:13<2:06:40, 29.79it/s]

  0%|          | 426/226834 [00:14<2:06:20, 29.87it/s]

  0%|          | 456/226834 [00:15<2:06:27, 29.83it/s]

  0%|          | 487/226834 [00:16<2:06:16, 29.87it/s]



KeyboardInterrupt: 

In [59]:
fixed_texts_df = pd.DataFrame(fixed_texts)

In [60]:
fixed_texts_df.sample(10)

,id,original_text,sentiment,text
39709,409901001601482752,"RT @jufyzatugade: Календарь ""2011, блеать"" К...",1,<__main__.Text object at 0x7f7de2e01668>
19535,409391359544418308,Мего крутой день))) прям весь день как праздни...,1,<__main__.Text object at 0x7f7de3c6a358>
4391,409055865749377024,RT @MAPCOKEMOB_MHE: @yulechka_minion @EnvoyPas...,1,<__main__.Text object at 0x7f7de3ff4dd8>
90609,410847118157676544,@chippolinka пасиба) Будем придумывать сценари...,1,<__main__.Text object at 0x7f7de45ebf98>
2405,408932134838341632,@itolmach у вас столько крутых деталей в доме!...,1,<__main__.Text object at 0x7f7de40484a8>
143062,412598177888358400,мля надо срочно где то найти объектив гелиос 4...,-1,<__main__.Text object at 0x7f7de2710898>
89113,410833740722896896,Мы готовы ко сну! Ахаха :) Спокойной ночи всем...,1,<__main__.Text object at 0x7f7de77ad400>
13466,409324774725529600,@ACostantinis мама !не тебе одной !я с ней с у...,1,<__main__.Text object at 0x7f7de3e52e80>
177531,416834399888744448,RT @Ackles_Collins: @xoxo_oleenom раз друзья н...,-1,<__main__.Text object at 0x7f7de1ff53c8>
122465,410039219768160257,"блин, если я пью чай без сахара, это не означа...",-1,<__main__.Text object at 0x7f7de2aa46d8>


# Exp

In [3]:
import requests

In [28]:
params = {'text': 'какого цвеета ты сегодя?', 'lang': 'ru,en'}
res = requests.get('http://speller.yandex.net/services/spellservice.json/checkText', params=params)

In [32]:
res.json()

[{'code': 1,
  'col': 7,
  'len': 6,
  'pos': 7,
  'row': 0,
  's': ['цвета'],
  'word': 'цвеета'},
 {'code': 1,
  'col': 17,
  'len': 6,
  'pos': 17,
  'row': 0,
  's': ['сегодня'],
  'word': 'сегодя'}]

In [33]:
t = 'какого цвеета ты сегодя?'

In [35]:
t[7:7+6]

'цвеета'